In [7]:
import os
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime
from sqlalchemy.orm import declarative_base, sessionmaker
from dotenv import load_dotenv
from langchain_cohere import ChatCohere
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from uuid import uuid4
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Set up SQLite database
DATABASE_URL = "sqlite:///C:/Users/jarmo/gits/interface_app_once_a_day/backend/chat_history.db"
Base = declarative_base()
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Define ChatMessage model
class ChatMessage(Base):
    __tablename__ = "chat_messages"
    id = Column(Integer, primary_key=True, index=True)
    session_id = Column(String, index=True)
    role = Column(String)
    content = Column(Text)
    timestamp = Column(DateTime, default=datetime.utcnow)

# Create the database tables
Base.metadata.create_all(bind=engine)

# Function to get a new database session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Function to save chat message to the database
def save_chat_message(session_id, role, content, db):
    db_message = ChatMessage(session_id=session_id, role=role, content=content)
    db.add(db_message)
    db.commit()
    db.refresh(db_message)
    return db_message

# Function to get chat message history from the database
def get_chat_message_history(session_id, db):
    return db.query(ChatMessage).filter(ChatMessage.session_id == session_id).all()

# Custom ChatMessageHistory class
class ChatMessageHistory:
    def __init__(self, messages=None):
        if messages is None:
            messages = []
        self.messages = messages

    def add_messages(self, new_messages):
        self.messages.extend(new_messages)

    def get_messages(self):
        return self.messages

# Initialize Cohere client
llm = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

# Define the interface prompt template
interface_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an llm interface app expert. You find produce suggestions for llm interface applications that is easy to make money with.",
        ),
        ("human", "{input}"),
    ]
)

# Create the interface chat chain
interface_chat = interface_prompt | llm | StrOutputParser()

# Define the cooking prompt template
cooking_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert of cooking easy to make food. You give easy to make recipes for people who are busy and don't have time to cook.",
        ),
        ("human", "{input}"),
    ]
)

# Create the cooking chat chain
cooking_chat = cooking_prompt | llm | StrOutputParser()

# Define tools
tools = [
    Tool.from_function(
        name="Interface Chat",
        description="For when you need to chat about interface apps. The question will be a string. Return in an easy to read structured form.",
        func=interface_chat.invoke,
    ),
    Tool.from_function(
        name="Cooking Chat",
        description="For when you need to chat about cooking. The question will be a string. Return in an easy to read structured form.",
        func=cooking_chat.invoke,
    ),
]

# Pull agent prompt from hub
agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Function to get chat message history for a session
def get_memory(session_id):
    db = next(get_db())
    messages = get_chat_message_history(session_id=session_id, db=db)
    # Convert database messages to a format suitable for ChatMessageHistory
    formatted_messages = [{"role": msg.role, "content": msg.content} for msg in messages]
    return ChatMessageHistory(formatted_messages)

# Create chat agent with message history
chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

# Main loop to handle user input and generate responses
while (q := input("> ")) != "exit":
    db = next(get_db())
    save_chat_message(SESSION_ID, "human", q, db)
    
    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID, "message_history": get_memory(SESSION_ID)}},
    )
    
    save_chat_message(SESSION_ID, "system", response["output"], db)
    print(response["output"])

c:\Users\jarmo\anaconda3\envs\learn_langchain-env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Session ID: ef515217-302b-41d1-8bae-64606af4f627
Here's a simple recipe for classic scrambled eggs, a great dish for beginner cooks:

**Ingredients:**
- 2-3 large eggs
- Milk or water (optional)
- Salt and pepper
- Butter

**Method:**
1. Beat the eggs with a fork, adding milk/water for a softer texture. Season with salt and pepper.
2. Melt butter in a pan over medium heat.
3. Pour in the eggs and scramble until cooked to your preference.
4. Serve with toast and your choice of toppings.

This recipe is a great starting point, and you can easily customize it by adding vegetables, meat, or spices. Enjoy your cooking journey!
